In [6]:
class Test:
    def __init__(self) -> None:
        print("in init")
        self.t = 1

    @classmethod
    def init_train(cls):
        obj = cls()
        return obj

In [7]:
test = Test.init_train()

in init


In [8]:
test.t

1

In [16]:
from modules.env import Environment, EnvironmentElement
from modules.history import History
from modules.agent_movement import Agent, Movement
from modules.reward import Reward
from modules.engine import Engine
from modules.qlearning import QLearning
from modules.policies import Boltzmann, EpsilonGreedy, Pursuit
from modules.losses import L2_Loss, L1_Loss
from modules.constants import *
from modules.viz import Visualization

import numpy as np

In [10]:
import pickle

with open("eval_hist_viz_test.pkl", "rb") as f:
    ehist = pickle.load(f)

In [36]:
history = ehist
metric_hist = history.get_metric_history()
avg_losses_per_ep = Visualization.get_avg_losses_per_ep(ehist)
avg_losses = [np.average(e['loss']) for e in avg_losses_per_ep]
best_ep = avg_losses_per_ep[np.argmin(avg_losses)]['episode']
b_random_agent_hist, b_random_env_hist = Visualization.get_random_episode(history, metric_hist, best_ep)

In [37]:
agent_history, env_history=  b_random_agent_hist, b_random_env_hist

In [39]:
grid_w, grid_h = env_history['occupancy'][0][0].shape
offsets = np.linspace(-0.3, 0.3, num=4)

# centers position to appear on the center of a grid instead of on grid lines
def center_grid_pos(pos, offset=0):
    x,y=pos
    return [x + 0.5 + offset, grid_h - y - 0.5 + offset]

# Collect relevant information about agent
agent_pos = {agent_idx: val['pos'] for agent_idx, val in agent_history.items()}
agent_info = {agent_idx: {"agent_type":val['agent_type'], "target_reached":val['target_reached']} \
    for agent_idx, val in agent_history.items() }
repeated_agent_pos = {agent_idx: [agent_pos[agent_idx][0]] \
    for agent_idx in agent_pos.keys()}
max_move_num = max([len(v) for v in agent_pos.values()])
done = []

for i in range(1, max_move_num):
    for agent_idx in range(4):
        if i >= len(agent_pos[agent_idx]) :
                continue
        agent_type, target_reached = agent_info[agent_idx].values()
        next_pos = agent_pos[agent_idx][i]
        repeated_agent_pos[agent_idx].append(next_pos)

        if (agent_type == EnvironmentElement.AGENT_TYPE_1 and len(target_reached)==2 and target_reached[-1] == i):
            other_agent = agent_history[agent_idx]['other_agent']
            if (repeated_agent_pos[other_agent][-1] == next_pos):
                done.append(agent_idx)
        elif (agent_type == EnvironmentElement.AGENT_TYPE_2 and len(target_reached) and target_reached[0]==i):
            other_agent = agent_history[agent_idx]['other_agent']
            if (repeated_agent_pos[other_agent][-1] == next_pos):
                done.append(other_agent)
        
        for repeat_agent_idx in range(4):
            if repeat_agent_idx == agent_idx or \
                i >= len(agent_pos[repeat_agent_idx]) or repeat_agent_idx in done:
                continue
            repeated_agent_pos[repeat_agent_idx].append(repeated_agent_pos[repeat_agent_idx][-1])
agent_pos = {agent_idx: [center_grid_pos(p, offset=offsets[agent_idx]) for p in val] for agent_idx, val in repeated_agent_pos.items()}
# get agent x and y positions throughout the episode
# for agent_idx in agent_pos.keys():
#     agent_idx_pos_x, agent_idx_pos_y = zip(*agent_pos[agent_idx])
#     agent_pos[agent_idx] = dict(agent_pos_x = list(agent_idx_pos_x),
#                                 agent_pos_y = list(agent_idx_pos_y))
    
# max_move_num = max([len(v['agent_pos_x']) for v in agent_pos.values()])
# loc_a = center_grid_pos(env_history['loc_a'])
# loc_b = env_history['loc_b']
# loc_b = center_grid_pos(loc_b)

In [40]:
agent_pos

{0: [[1.2, 1.2],
  [1.2, 2.2],
  [1.2, 2.2],
  [1.2, 2.2],
  [1.2, 2.2],
  [1.2, 3.2],
  [1.2, 3.2],
  [1.2, 3.2],
  [1.2, 3.2],
  [0.2, 3.2],
  [0.2, 3.2],
  [0.2, 3.2],
  [0.2, 3.2],
  [0.2, 4.2],
  [0.2, 4.2],
  [0.2, 4.2],
  [0.2, 4.2],
  [1.2, 4.2],
  [1.2, 4.2],
  [1.2, 4.2],
  [1.2, 4.2],
  [2.2, 4.2]],
 1: [[2.4, 3.4],
  [2.4, 3.4],
  [1.4, 3.4],
  [1.4, 3.4],
  [1.4, 3.4],
  [1.4, 3.4],
  [0.4, 3.4],
  [0.4, 3.4],
  [0.4, 3.4],
  [0.4, 3.4],
  [0.4, 4.4],
  [0.4, 4.4],
  [0.4, 4.4],
  [0.4, 4.4],
  [1.4, 4.4],
  [1.4, 4.4],
  [1.4, 4.4],
  [1.4, 4.4],
  [2.4, 4.4]],
 2: [[2.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [2.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [3.6, 4.6],
  [4.6, 4.6],
  [4.6, 4.6],
  [4.6, 3.6],
  [4.6, 3.6],
  [4.6, 2.6],
  [4.6, 2.6],
  [4.6, 1.6],
  [4.6, 1

In [26]:
agent_info

{0: {'agent_type': <EnvironmentElement.AGENT_TYPE_1: 1>,
  'target_reached': [5, 7]},
 1: {'agent_type': <EnvironmentElement.AGENT_TYPE_1: 1>,
  'target_reached': [3, 5]},
 2: {'agent_type': <EnvironmentElement.AGENT_TYPE_2: 2>,
  'target_reached': [4, 10]},
 3: {'agent_type': <EnvironmentElement.AGENT_TYPE_2: 2>,
  'target_reached': [6, 12]}}

In [42]:
item_pos = dict()

for agent_idx in agent_pos.keys():
    agent_type = agent_info[agent_idx]['agent_type']
    if agent_type == EnvironmentElement.AGENT_TYPE_2:
        continue
    picked_up_item = False
    item_pos_agent = []
    for (agent_x, agent_y) in agent_pos[agent_idx]:
        # print(agent_x, agent_y)
        if picked_up_item == False:
            print()
            if (0.0 <= agent_x < 1.0 and 4.0 <= agent_y < 5.0):
                picked_up_item = True

        if picked_up_item == True:
            item_pos_agent.append([agent_x + 0.05, agent_y+0.05])
    item_pos[agent_idx] = item_pos_agent
    

In [43]:
item_pos

{0: [[0.25, 4.15],
  [0.25, 4.15],
  [0.25, 4.15],
  [0.25, 4.15],
  [1.25, 4.15],
  [1.25, 4.15],
  [1.25, 4.15],
  [1.25, 4.15],
  [2.25, 4.15]],
 1: [[0.45, 4.3500000000000005],
  [0.45, 4.3500000000000005],
  [0.45, 4.3500000000000005],
  [0.45, 4.3500000000000005],
  [1.45, 4.3500000000000005],
  [1.45, 4.3500000000000005],
  [1.45, 4.3500000000000005],
  [1.45, 4.3500000000000005],
  [2.4499999999999997, 4.3500000000000005]]}

In [25]:
agent_pos

{0: {'agent_pos_x': [3.2,
   2.2,
   2.2,
   2.2,
   2.2,
   2.2,
   2.2,
   2.2,
   2.2,
   1.2,
   1.2,
   1.2,
   1.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   0.2,
   1.2,
   1.2,
   1.2,
   2.2],
  'agent_pos_y': [2.2,
   2.2,
   2.2,
   2.2,
   2.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   3.2,
   4.2,
   4.2,
   4.2,
   4.2,
   4.2,
   4.2,
   4.2,
   4.2]},
 1: {'agent_pos_x': [0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   0.4,
   1.4,
   1.4,
   1.4,
   1.4,
   2.4],
  'agent_pos_y': [1.4,
   1.4,
   2.4,
   2.4,
   2.4,
   2.4,
   3.4,
   3.4,
   3.4,
   3.4,
   4.4,
   4.4,
   4.4,
   4.4,
   4.4,
   4.4,
   4.4,
   4.4,
   4.4]},
 2: {'agent_pos_x': [3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   3.6,
   2.6,
   2.6,
   2.6,
   2.6,
   3.6,
   3.6,
   3.6,
   4.6,
   4.6,
   4.6,
   4.6

---

In [2]:
history = History()
env = Environment(GRID_SIZE, history, loc_b=LOC_B)
agent = Agent(env, history)
reward = Reward(env, agent)

In [3]:
learning = QLearning()

# policy = EpsilonGreedy(1, learning=learning)
policy = Boltzmann(1000, learning=learning)
# policy = Pursuit(0.1)
# policy = MonteCarlo(learning=learning)

learning.policy = policy

In [9]:
engine = Engine(env, reward, agent, learning, history, iters=50)
engine.run()

Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!
Agent has picked up item at location A!


In [10]:
%matplotlib inline

In [11]:
viz = Visualization(env, learning, history)

In [12]:
viz.plot_agent_path()